In [ ]:
import numpy as np
import os
import glob
import nibabel as nib
import sys
import json
import matplotlib.pyplot as plt

src.resampling을 이용해서 다음과 같이 리샘플링함

[resample_fixedsize_fixedspacing(nii_file, './result', sitk.sitkLinear, 128, [1,1,1]) for nii_file in nii_list]

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\ai6\Desktop\git\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\src\preprocessing.py').resolve().parent.parent))
sys.path.insert(0, str(Path(r'C:\Users\ai6\Desktop\git\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\src\helper.py').resolve().parent.parent))

In [ ]:
from src.helper import *
from src.eda_nifti import *

In [ ]:
nii_list = recursive_find_all_files(r'C:\Users\ai6\Desktop\git\ANALYSIS-NIFTI-FOR-DEEP-LEARNING\notebooks\10.legion_boundary_difference\result', 'nii.gz')

In [ ]:
nii_list

In [ ]:
matching_path_dict = get_matching_path_dict(nii_list, 'msk')

In [ ]:
matching_path_dict_keys = [key for key in matching_path_dict.keys()]

In [ ]:
######### 1) 테스트 타겟: adc

In [ ]:
matching_path_dict_keys[0]

In [ ]:
########## 2) 테스트 라벨: msk

In [ ]:
matching_path_dict[matching_path_dict_keys[0]]

In [ ]:
adc_nii = nib.load(matching_path_dict_keys[0])
adc_array = adc_nii.get_fdata()

In [ ]:
adc_array.shape

In [ ]:
mask_nii = nib.load(matching_path_dict[matching_path_dict_keys[0]])
mask_array = mask_nii.get_fdata()

In [ ]:
mask_array.shape

In [ ]:
!pip install ipywidgets

In [ ]:
from src.registration_metric import *

In [ ]:
reg_check = RegistrationMetric()

In [ ]:
reg_check.visuall_validation(matching_path_dict_keys[0], matching_path_dict[matching_path_dict_keys[0]])

In [ ]:
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

In [ ]:
fixed_image = sitk.ReadImage(matching_path_dict_keys[0])
moving_image = sitk.ReadImage(matching_path_dict[matching_path_dict_keys[0]])

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_image))